## 동영상 관련 이론


`VideoFileClip` 은 비디오 파일을 읽어올 때 사용되는 클래스
`ipython_display()` 는 동영상을 주피터 노트북에 렌더링할 수 있게 도와주는 함수

동영상을 처리하기 위해서는

1. 동영상을 읽고
2. 프레임별 이미지를 `numpy` 형태로 추출하고
3. 얼굴 검출 같은 이미지 처리를 수행하고
4. 다시 동영상 등으로 조합
해야 한다.

`moviepy` 는 동영상을 읽는 것은 쉽지만 `NumPy` 데이터로 변환하기 위한 과정을 거쳐야 하고, 프레임 이미지에 검출 모델을 적용하더라도, 결과 동영상을 확인하기 위해서는 동영상 파일을 저장했다 다시 읽어야 하기 때문이다.

OpenCV에서는 `getTickCount()` 와 `getTickFrequency()` 를 사용해서 시간을 측정한다. 초 단위로 나오기 때문에 보통 이미지 한 장을 처리할 때는 1,000을 곱해 ms 단위로 프로그램 속도를 관찰한다. (보통 비디오는 30fps이기 때문에 프레임당 ms 처리 단위를 가지기 때문이다)

### `OpenCV`로 동영상을 저장하는 방법

아래 코드를 아래쪽에 붙여넣은 후 파이썬 파일을 실행하면 저장이 됨

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter('./images/result.mp4', fourcc, 30, (1280, 720))

`img2sticker_orig()` 함수는 크게 다섯 단계로 나눠진다.

1. 전처리 (Preprocess)
2. 얼굴 검출 (Detection)
3. 랜드마크 검출 (Landmark)
4. 좌표변환 (Coord)
5. 스티커 합성 (Sticker)

### `dlib`의 특징

`dlib` 얼굴 검출기(face detector)는 **HOG(histogram of oriented gradient) 기반 알고리즘**을 사용

- [기계 학습(Machine Learning, 머신 러닝)은 즐겁다! Part 4](https://medium.com/@jongdae.lim/%EA%B8%B0%EA%B3%84-%ED%95%99%EC%8A%B5-machine-learning-%EC%9D%80-%EC%A6%90%EA%B2%81%EB%8B%A4-part-4-63ed781eee3c)

`dlib`은 HOG 특징 공간에서 **슬라이딩 윈도우(sliding window)** 기법을 통해 얼굴을 검출


- [Sliding Windows for Object Detection with Python and OpenCV - PyImageSearch](https://www.pyimagesearch.com/2015/03/23/sliding-windows-for-object-detection-with-python-and-opencv/)

HOG 특성 맵(feature map)에서 입력 이미지 크기(HD: 1280x720)만큼 슬라이딩 윈도우를 수행하기 때문에 프로그램은 $O(1280 * 720 * (bbox~size) * 피라미드~개수) = O(n^3)$의 시간을 가지므로 느려질 수밖에 없다.

속도 개선 방법 중 가장 간단한 방법은 사용되는 이미지 크기를 줄이거나 피라미드 수를 줄이는 방법


## 신호와 신호처리

떨리는 스티커를 어떻게 안정화할 수 있을까요?

먼저 신호에 대해 알아봅시다.

### 신호(Signal)와 노이즈(Noise)
---
시간과 공간에 따라 변화하는 물리량을 나타내는 함수를 **신호(signal)**

신호에 노이즈(noise)가 섞이는 이유는 크게 두 가지

1. 신호를 출력하는 모델의 노이즈
2. 신호를 측정할 때 생기는 노이즈

사람이 뛰는 속도 측정을 예시로 들자면

1. 뛰는 사람이 힘들어서 `19km/h` 로 뛰다가 다시 `21km/h` 로 뛰는 경우
2. 측정 카메라가 사람 다리를 측정할 때, 머리를 측정할 때 속도가 달라지는 경우


### 신호처리(Signal Processing)
---
원본 신호를  원하는 신호의 형태로 만드는 방법을 **신호처리(signal processing)** 


## 칼만 필터

- **칼만 필터(Kalman filter)** 는 시스템 모델과 측정 모델을 만들고 데이터 입력을 바탕으로 각 모델을 예측하는 알고리즘
- 예측된 모델을 바탕으로 현재의 실제 값을 추정할 수 있고 다음 시점의 모델 출력을 예측할 수 있다. 
- 이 때 시스템 모델과 측정 모델은 모두 선형이고(linear)하고 가우시안(gaussian) 분포를 따르는 경우를 가정

- [Kalman filter 소개](https://medium.com/@celinachild/kalman-filter-%EC%86%8C%EA%B0%9C-395c2016b4d6)

![image](https://user-images.githubusercontent.com/70703320/116035982-e6144b80-a6a0-11eb-9b85-12bc6cb80be6.png)
\[ 칼만 필터 플로우 차트 \]

- **얼굴 검출(detection)** 과 **얼굴 랜드마크(landmark)** 는 프레임마다 가우시안 오차를 갖는 측정 시스템이라고 가정할 수 있다. 
- 이는 라벨링 할 때 사람도 매번 같은 위치를 찍을 수 없으며 자연 상태의 측정값은 대체로 정규분포를 따르기 때문

**얼굴 랜드마크에 칼만 필터를 적용하면 안정적인 스티커 결과를 얻을 수 있을 것을 예상할 수 있다.**

칼만 필터를 구현하는 `kalman_filter()` 함수

In [4]:
def kalman_filter(z, x, P):
# Kalman Filter Algorithm
    # 예측 단계
    xp = A @ x
    Pp = A @ P @ A.T + Q

    # 추정 단계
    K = Pp @ H.T @ inv(H @ Pp @ H.T + R)
    x = xp + K @ (z - H @ xp)
    P = Pp - K @ H @ Pp
    return x, P